<a href="https://cognitiveclass.ai"><img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/IDSNlogo.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

In this lab, we will learn how to convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in New York City. We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the _k_-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

In [3]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 122kB 6.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         235 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda

In [4]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html=requests.get(url).text

In [5]:
soup = BeautifulSoup(html, 'html5lib')

In [6]:
keys = (['PostalCode', 'Borough', 'Neighborhood'])
dict.fromkeys(keys, [])

{'PostalCode': [], 'Borough': [], 'Neighborhood': []}

In [7]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [72]:
df.sort_index
df

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
df.shape

(103, 3)

In [32]:
url2 ="https://cocl.us/Geospatial_data"
df1 = pd.read_csv(url2)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [78]:
neighborhoods = pd.concat([df, df1], axis=1, join="inner")
neighborhoods.drop(['Postal Code'], axis=1, inplace= True)

In [82]:
neighborhoods.sort_index
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
79,M4S,Central Toronto,Davisville,43.713756,-79.490074
67,M4P,Central Toronto,Davisville North,43.653206,-79.400049
61,M4N,Central Toronto,Lawrence Park,43.648198,-79.379817
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.648960,-79.456325
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.636966,-79.615819
68,M5P,Central Toronto,Forest Hill North & West,43.628947,-79.394420
73,M4R,Central Toronto,North Toronto West,43.693781,-79.428191
62,M5N,Central Toronto,Roselawn,43.733283,-79.419750
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.689026,-79.453512
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.756303,-79.565963


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [83]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [84]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I have decided to explore Downtown Toronto area. 

In [53]:
downtown_toronto_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.756303,-79.565963
2,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


In [54]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [55]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = '3NFT2XQDSZUZOYKQHYNDI23TOFBATXRUB4WESMWLGV0PHOWI' # your Foursquare ID
CLIENT_SECRET = 'DWWN5OSAYBLSB1YPJUAVUEAWUPOP3LHNTFHL4UJLJJUZA3RF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3NFT2XQDSZUZOYKQHYNDI23TOFBATXRUB4WESMWLGV0PHOWI
CLIENT_SECRET:DWWN5OSAYBLSB1YPJUAVUEAWUPOP3LHNTFHL4UJLJJUZA3RF


Get the neighborhood's name.

In [57]:
downtown_toronto_data.loc[0, 'Neighborhood']

'Rosedale'

Get the neighborhood's latitude and longitude values.

In [85]:
neighborhood_latitude = downtown_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6362579, -79.49850909999999.


#### Now, let's get the top 100 venues that are in Downtown Toronto within a radius of 1000 meters.
First, let's create the GET request URL. Name your URL **url**.

In [94]:
LIMIT = 100
radius = 2000
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url3

'https://api.foursquare.com/v2/venues/explore?&client_id=3NFT2XQDSZUZOYKQHYNDI23TOFBATXRUB4WESMWLGV0PHOWI&client_secret=DWWN5OSAYBLSB1YPJUAVUEAWUPOP3LHNTFHL4UJLJJUZA3RF&v=20180605&ll=43.6362579,-79.49850909999999&radius=2000&limit=100'

Send the GET request and examine the resutls

In [95]:
results = requests.get(url3).json()
results

{'meta': {'code': 200, 'requestId': '60721982c0166e7a8c929f3e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Stonegate - Queensway',
  'headerFullLocation': 'Stonegate - Queensway, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 139,
  'suggestedBounds': {'ne': {'lat': 43.65425791800001,
    'lng': -79.47368454105066},
   'sw': {'lat': 43.61825788199998, 'lng': -79.52333365894933}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adf75b3f964a520c87a21e3',
       'name': "Tom's Dairy Freeze",
       'location': {'address': '630 The Queensway',
        'crossStreet': 'Mcintosh Ave',
        'lat': 43.627897,
        'lng': -79.496638,
        'labeledL

In [96]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [97]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Tom's Dairy Freeze,Ice Cream Shop,43.627897,-79.496638
1,Starbucks,Coffee Shop,43.627931,-79.489286
2,Starbucks,Coffee Shop,43.624654,-79.508217
3,Bua Thai,Thai Restaurant,43.626176,-79.503228
4,Mamma Martino's,Italian Restaurant,43.627899,-79.496052


And how many venues were returned by Foursquare?

In [98]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [100]:
downtown_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'] )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Christie


In [102]:
print(downtown_venues.shape)
downtown_venues.head()

(480, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.636258,-79.498509,Tom's Dairy Freeze,43.627897,-79.496638,Ice Cream Shop
1,Rosedale,43.636258,-79.498509,Mamma Martino's,43.627899,-79.496052,Italian Restaurant
2,Rosedale,43.636258,-79.498509,Izba Schnitzel House,43.627567,-79.497410,Eastern European Restaurant
3,Rosedale,43.636258,-79.498509,Park Lawn Park,43.642014,-79.491153,Park
4,Rosedale,43.636258,-79.498509,Jeff Healey Park,43.630397,-79.495214,Park


In [103]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,4,4,4,4,4,4
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",47,47,47,47,47,47
Central Bay Street,13,13,13,13,13,13
Christie,29,29,29,29,29,29
Church and Wellesley,15,15,15,15,15,15
"Commerce Court, Victoria Hotel",58,58,58,58,58,58
"First Canadian Place, Underground city",9,9,9,9,9,9
"Garden District, Ryerson",14,14,14,14,14,14
"Harbourfront East, Union Station, Toronto Islands",28,28,28,28,28,28


In [105]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 139 uniques categories.


In [107]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,ATM,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Store,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Coffee Shop,College Stadium,Comic Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Dry Cleaner,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Market,Metro Station,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Moving Target,Neighborhood,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Park,Pastry Shop,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Pool,Pub,Restaurant,River,Salad Place,Sandwich Place,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Spa,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tennis Court,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,ATM,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Store,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chiropractor,Coffee Shop,College Stadium,Comic Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Dry Cleaner,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Market,Metro Station,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Moving Target,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Park,Pastry Shop,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Pool,Pub,Restaurant,River,Salad Place,Sandwich Place,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Spa,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tennis Court,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.021277,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.021277,0.000000,0.021277,0.000000,0.021277,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.063830,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063830,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.021277,0.042553,0.021277,0.021277,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.021277,0.000000,0.000000,0.021277,0.000000,0.000000,0.021277,0.0

In [109]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0                       Park  0.75
1                       Pool  0.25
2  Middle Eastern Restaurant  0.00
3  Latin American Restaurant  0.00
4            Laundry Service  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0                Park  0.09
1             Brewery  0.06
2         Coffee Shop  0.06
3         Pizza Place  0.06
4  Italian Restaurant  0.04


----Central Bay Street----
                         venue  freq
0                     Pharmacy  0.15
1                       Bakery  0.08
2        Entertainment Service  0.08
3  Eastern European Restaurant  0.08
4            Convenience Store  0.08


----Christie----
               venue  freq
0               Park  0.10
1      Shopping Mall  0.07
2           Bus Stop  0.07
3           Pharmacy  0.07
4  Food & Drink Shop  0.03


----Church and Wellesley----
                venue 

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Park,Pool,Convenience Store,Discount Store,Diner,Dessert Shop,Dance Studio,Curling Ice,Cosmetics Shop,Comic Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Coffee Shop,Pizza Place,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Burrito Place,Harbor / Marina,Ice Cream Shop
2,Central Bay Street,Pharmacy,Park,Pizza Place,Eastern European Restaurant,Entertainment Service,Bakery,Bus Line,Baby Store,Supermarket,Convenience Store
3,Christie,Park,Bus Stop,Pharmacy,Shopping Mall,Grocery Store,Café,Chinese Restaurant,Laundry Service,Coffee Shop,Intersection
4,Church and Wellesley,Gas Station,Pizza Place,Chinese Restaurant,Supermarket,Intersection,Breakfast Spot,Ice Cream Shop,Discount Store,Flea Market,Golf Driving Range


## Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [112]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 2, 0, 0, 2, 2, 0, 0], dtype=int32)

In [113]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.636258,-79.498509,4,Park,Gym / Fitness Center,Bus Stop,Italian Restaurant,Shopping Mall,Ice Cream Shop,Eastern European Restaurant,Flower Shop,Fried Chicken Joint,Chinese Restaurant
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.756303,-79.565963,0,Park,Pizza Place,Electronics Store,Italian Restaurant,Bank,Bakery,Shopping Mall,Pharmacy,College Stadium,Comic Shop
2,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242,0,Gas Station,Pizza Place,Chinese Restaurant,Supermarket,Intersection,Breakfast Spot,Ice Cream Shop,Discount Store,Flea Market,Golf Driving Range
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Pizza Place,Bank,Restaurant,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Sports Bar,Supermarket,Greek Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,2,Park,General Entertainment,College Stadium,Skating Rink,Ice Cream Shop,Restaurant,Gym Pool,Gym,Convenience Store,Thai Restaurant


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [115]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Park,Pizza Place,Electronics Store,Italian Restaurant,Bank,Bakery,Shopping Mall,Pharmacy,College Stadium,Comic Shop
2,Downtown Toronto,0,Gas Station,Pizza Place,Chinese Restaurant,Supermarket,Intersection,Breakfast Spot,Ice Cream Shop,Discount Store,Flea Market,Golf Driving Range
3,Downtown Toronto,0,Pizza Place,Bank,Restaurant,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Sports Bar,Supermarket,Greek Restaurant
5,Downtown Toronto,0,Chinese Restaurant,Bakery,Coffee Shop,Pizza Place,Intersection,Fast Food Restaurant,Bank,Other Great Outdoors,Pharmacy,Breakfast Spot
8,Downtown Toronto,0,Coffee Shop,Turkish Restaurant,Vietnamese Restaurant,Sandwich Place,Gas Station,Electronics Store,Gym,Intersection,Italian Restaurant,Latin American Restaurant
9,Downtown Toronto,0,Coffee Shop,Skating Rink,Café,Park,Thai Restaurant,Pizza Place,Sandwich Place,Beer Store,Bus Stop,Pastry Shop
10,Downtown Toronto,0,Indian Restaurant,Park,Coffee Shop,Restaurant,Fast Food Restaurant,Beach,Brewery,Bakery,Café,Harbor / Marina
11,Downtown Toronto,0,Coffee Shop,Grocery Store,Italian Restaurant,Gym,Park,Thai Restaurant,Sushi Restaurant,Bagel Shop,Bank,Restaurant
13,Downtown Toronto,0,Coffee Shop,Café,Bakery,Italian Restaurant,Pizza Place,Pub,Restaurant,Bank,Bar,Sushi Restaurant
14,Downtown Toronto,0,Park,Coffee Shop,Pizza Place,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Burrito Place,Harbor / Marina,Ice Cream Shop
